### TODO
- tc model for two variables with no correlated erros - use state space inference library
- estimate on simulated data
- estimate on real data
- add var error correlation
- add trend correlation
- extedn to more variables

In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner/_wip/TCVAR")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner\_wip\TCVAR`


In [2]:
Pkg.add(url="https://github.com/TuringLang/SSMProblems.jl", subdir="GeneralisedFilters")
Pkg.add(url="https://github.com/TuringLang/SSMProblems.jl", subdir="SSMProblems")

    Updating git-repo `https://github.com/TuringLang/SSMProblems.jl`
    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner\_wip\TCVAR\Project.toml`
  No Changes to `C:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner\_wip\TCVAR\Manifest.toml`
    Updating git-repo `https://github.com/TuringLang/SSMProblems.jl`
   Resolving package versions...
  No Changes to `C:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner\_wip\TCVAR\Project.toml`
  No Changes to `C:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner\_wip\TCVAR\Manifest.toml`


In [3]:
using Revise
using LinearAlgebra
using SSMProblems
using GeneralisedFilters
using Turing

includet("TCVAR_model.jl")

In [5]:
#params_true = NamedTuple{(:στπ, :στr, :Φ11, :Φ12, :Φ21, :Φ22, :σcπ, :σcr)}((.02, .01, .6, .3, .4, .2, .015, .01))
   

In [7]:
 # True parameters (for simulation)
phi_true = [0.8 0.1; -0.1 0.7]
Qcycle_true = [0.05 0.01; 0.01 0.08]
Qtrend_true = [0.0004 0.0; 0.0 0.0004]

true_tcvar_model = TCVAR(phi_true, Qtrend_true, Qcycle_true)

initial, states, test_obs = sample(true_tcvar_model, 100)



([0.37241329959417646, -10.632027304110208, 4.900613825812982, 6.615186528283386], [[0.3840207614389971, -10.669316243028119, 4.4743818517950364, 4.323582372534172], [0.41120751656382526, -10.684102375973794, 4.242209452719639, 2.2480713297427775], [0.39816039726788804, -10.69025682326522, 3.718355473948895, 1.0372804653220185], [0.4172285555540968, -10.662743622846683, 3.133799777897864, 0.6212285994440825], [0.42422844133644266, -10.644816295707823, 2.454050650538267, 0.651297783199887], [0.42574605961243517, -10.633908102856994, 2.0768189438307614, -0.13785102188912812], [0.4213607525609648, -10.640892972879449, 2.067699090862746, -0.48207734796595725], [0.41028299409271, -10.649738108447709, 1.157104108705836, -0.7523921828996911], [0.41234307019289573, -10.668839332048499, 0.86795726673427, -0.925597843783811], [0.4050186222741935, -10.68529876968966, 0.6206816376375864, -1.0383242464749427]  …  [0.14594753375704372, -10.97730075392246, -0.18153627944290684, -0.740619537775532], [

In [8]:
model = trendcycle_model(test_obs)

n_samples = 500
chain = sample(model, NUTS(), n_samples)

Sampling   0%|█                                         |  ETA: N/A
Sampling 100%|██████████████████████████████████████████| Time: 0:00:01


MethodError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{DynamicPPL.DynamicPPLTag, Float64}, Float64, 12})
The type `Float64` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  (::Type{T})(::Real, !Matched::RoundingMode) where T<:AbstractFloat
   @ Base rounding.jl:265
  (::Type{T})(::T) where T<:Number
   @ Core boot.jl:900
  Float64(!Matched::IrrationalConstants.Logπ)
   @ IrrationalConstants C:\Users\matsz\.julia\packages\IrrationalConstants\lWTip\src\macro.jl:131
  ...


In [16]:
priors = (
    initial_trend_mean = 0.03,
    initial_trend_std = 0.09,
    initial_cycle = (mean=0., 
                     std = .02)

)

priors.initial_cycle.std

0.02